In [76]:
import pandas as pd
from lxml import etree
import re


#on ouvre le fichier XML

xml_file = '/Users/margotpascual/Documents/ENC/dico-topo/data/PO_t7/PO_t7.xml'
xml_tree = etree.parse(xml_file)

#xpath pour sélectionner tous les articles + regex pour s'occuper des balises XML

xpath_expr = '//article'

pattern = re.compile(r'<.*?>')

#on crée une liste vide pour ensuite mettre toutes les valeurs nécessaires au dataframe dans un dictionnaire par la suite

data = []

#on s'occupe des vedettes: pour l'instant, notre code ne prend pas en compte les vedettes avec + d'une valeur. A voir
#si il faut changer ça. sur le xpath cependant, on dirait qu'il n'y en a pas ?
for article in xml_tree.xpath(xpath_expr):

#on enleve les localisationpa : inutiles pour nous si on ne se focalise que sur la france.
        if article.find('./localisationpa') is not None:
             continue
        vedette_i = article.xpath('./vedette/i')
        vedette_text = vedette_i[0].text if vedette_i else ''

#pareil: il y a un seul localisationde en double, mais il ne le prend pas en compte. A voir si il faut changer ça
        localisationde = article.find('./localisationde')
        localisationde_text = localisationde.text if localisationde is not None else 'none'

#idem. autre version du code qui en prend deux et sépare par ',' existe, à voir si c'est vraiment pertinent.
        localisationca = article.find('./localisationca')
        if localisationca is not None:
            localisationca_text = re.search(r'<localisationca>(.*?)<\/localisationca>', etree.tostring(localisationca, encoding=str)).group(1)
            localisationca_text = localisationca_text.replace('c<sup>on</sup>', 'canton')
        else:
            localisationca_text = 'none'

#idem. Il n'y en a qu'un avec + d'une localisation -> l'autre étant en suisse. Utile ou pas? à voir.
        localisationco = article.find('./localisationco')
        if localisationco is not None:
            localisationco_text = re.search(r'<localisationco>(.*?)<\/localisationco>', etree.tostring(localisationco, encoding=str)).group(1)
            localisationco_text = localisationco_text.replace('c<sup>ne</sup>', 'commune')
        else: 
            localisationco_text = 'none'
    
#on insère tout dans un dictionnaire        
        data.append({
            'old-id': article.get('old-id'),
            'vedette': vedette_text,
            'localisationde': localisationde_text,
            'localisationca': localisationca_text,
            'localisationco': localisationco_text
        })


In [26]:
#on crée notre dataframe

df = pd.DataFrame(data)

In [27]:
df.head(5)

,old-id,vedette,localisationde,localisationca,localisationco
0,PO7-00001,Abbaye,Drôme,canton de Die,commune de Die
1,PO7-00002,Abbaye,Isère,canton de Roybon,commune de Marnans
2,PO7-00003,Abbaye,Haute-Savoie,canton du Biot,commune de Saint-Jean-d’Aulph
3,PO7-00005,Abbaye-d’Acey,Jura,canton de Gendrey,commune d’Ougney
4,PO7-00006,Abbaye-de-Grandvaux,Jura,canton de Saint-Laurent,commune de Rivière-Devant


In [28]:
print(df)

         old-id              vedette localisationde           localisationca  \
0     PO7-00001               Abbaye          Drôme            canton de Die   
1     PO7-00002               Abbaye          Isère         canton de Roybon   
2     PO7-00003               Abbaye   Haute-Savoie           canton du Biot   
3     PO7-00005        Abbaye-d’Acey           Jura        canton de Gendrey   
4     PO7-00006  Abbaye-de-Grandvaux           Jura  canton de Saint-Laurent   
...         ...                  ...            ...                      ...   
3483  PO7-04280           Zaessingue      Haut-Rhin        canton de Landser   
3484  PO7-04282           Zellenberg      Haut-Rhin    canton de Kaysersberg   
3485  PO7-04285           Zillisheim      Haut-Rhin       canton de Mulhouse   
3486  PO7-04286           Zimmerbach      Haut-Rhin    canton de Wintzenheim   
3487  PO7-04287          Zimmersheim      Haut-Rhin        canton d’Habsheim   

                     localisationco  
0

In [29]:
departements_df = pd.read_excel('departements-francais.xls')

# On crée un dictionnaire avec les éléments qui nous intéressent de notre spreadsheet du début
departements_dict = dict(zip(departements_df['NOM'], departements_df['NUMÉRO']))

# on ajoute notre colonne avec les numéros des départements qui matchent
df['departement_numero'] = df['localisationde'].apply(lambda x: departements_dict.get(x, 'none'))

print(df)

         old-id              vedette localisationde           localisationca  \
0     PO7-00001               Abbaye          Drôme            canton de Die   
1     PO7-00002               Abbaye          Isère         canton de Roybon   
2     PO7-00003               Abbaye   Haute-Savoie           canton du Biot   
3     PO7-00005        Abbaye-d’Acey           Jura        canton de Gendrey   
4     PO7-00006  Abbaye-de-Grandvaux           Jura  canton de Saint-Laurent   
...         ...                  ...            ...                      ...   
3483  PO7-04280           Zaessingue      Haut-Rhin        canton de Landser   
3484  PO7-04282           Zellenberg      Haut-Rhin    canton de Kaysersberg   
3485  PO7-04285           Zillisheim      Haut-Rhin       canton de Mulhouse   
3486  PO7-04286           Zimmerbach      Haut-Rhin    canton de Wintzenheim   
3487  PO7-04287          Zimmersheim      Haut-Rhin        canton d’Habsheim   

                     localisationco dep

In [30]:
df = df.rename(columns={'old-id' : 'article_id', 'localisationca' : 'canton_code', 'localisationco_present' : 'nom_commune', 'departement_numero' : 'dpt_code'})

In [31]:
df

,article_id,vedette,localisationde,canton_code,localisationco,dpt_code
0,PO7-00001,Abbaye,Drôme,canton de Die,commune de Die,26
1,PO7-00002,Abbaye,Isère,canton de Roybon,commune de Marnans,38
2,PO7-00003,Abbaye,Haute-Savoie,canton du Biot,commune de Saint-Jean-d’Aulph,74
3,PO7-00005,Abbaye-d’Acey,Jura,canton de Gendrey,commune d’Ougney,39
4,PO7-00006,Abbaye-de-Grandvaux,Jura,canton de Saint-Laurent,commune de Rivière-Devant,39
...,...,...,...,...,...,...
3483,PO7-04280,Zaessingue,Haut-Rhin,canton de Landser,none,68
3484,PO7-04282,Zellenberg,Haut-Rhin,canton de Kaysersberg,none,68
3485,PO7-04285,Zillisheim,Haut-Rhin,canton de Mulhouse,none,68
3486,PO7-04286,Zimmerbach,Haut-Rhin,canton de Wintzenheim,none,68


In [32]:
df = df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco']]

In [33]:
df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco
0,PO7-00001,Abbaye,Drôme,26,canton de Die,commune de Die
1,PO7-00002,Abbaye,Isère,38,canton de Roybon,commune de Marnans
2,PO7-00003,Abbaye,Haute-Savoie,74,canton du Biot,commune de Saint-Jean-d’Aulph
3,PO7-00005,Abbaye-d’Acey,Jura,39,canton de Gendrey,commune d’Ougney
4,PO7-00006,Abbaye-de-Grandvaux,Jura,39,canton de Saint-Laurent,commune de Rivière-Devant
...,...,...,...,...,...,...
3483,PO7-04280,Zaessingue,Haut-Rhin,68,canton de Landser,none
3484,PO7-04282,Zellenberg,Haut-Rhin,68,canton de Kaysersberg,none
3485,PO7-04285,Zillisheim,Haut-Rhin,68,canton de Mulhouse,none
3486,PO7-04286,Zimmerbach,Haut-Rhin,68,canton de Wintzenheim,none


In [34]:
df.to_csv("my_dataframe_entier.csv", index=False)

In [61]:
#on ne garde que les lieux sans localisationco, qu'on pense à ce stade être des communes.

filtered_df = df[df['localisationco'] == 'none']
filtered_df['dpt_code'] = filtered_df['dpt_code'].apply(lambda x: 'DEP_' + str(x) if x != 'none' else x)

print(filtered_df)

     article_id      vedette localisationde dpt_code  \
7     PO7-00009     Abbenans          Doubs   DEP_25   
8     PO7-00010  Abbevillers          Doubs   DEP_25   
9     PO7-00011    Aboncourt    Haute-Saône   DEP_70   
10    PO7-00012    Abondance   Haute-Savoie   DEP_74   
11    PO7-00013       Abrets          Isère   DEP_38   
...         ...          ...            ...      ...   
3483  PO7-04280   Zaessingue      Haut-Rhin   DEP_68   
3484  PO7-04282   Zellenberg      Haut-Rhin   DEP_68   
3485  PO7-04285   Zillisheim      Haut-Rhin   DEP_68   
3486  PO7-04286   Zimmerbach      Haut-Rhin   DEP_68   
3487  PO7-04287  Zimmersheim      Haut-Rhin   DEP_68   

                       canton_code localisationco  
7              canton de Rougemont           none  
8            canton d’Hérimoncourt           none  
9        canton de Combeaufontaine           none  
10                            none           none  
11    canton du Pont-de-Beauvoisin           none  
...            

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_9168/3304469425.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['dpt_code'] = filtered_df['dpt_code'].apply(lambda x: 'DEP_' + str(x) if x != 'none' else x)


In [62]:
#on a check; le nombre correspond bien au nombre sur le XML. On save.

filtered_df.to_csv("my_dataframe_filtree.csv", index=False)

In [73]:
#on regarde combien d'articles n'ont pas de dpt_code dans ceux sans localisationco ; on devra les soustraire au total
#des localisationco sans insee_id car ils n'auraient pas pu être liés de toute manière.

result = filtered_df[filtered_df['dpt_code'] == 'none']

print(result)

     article_id    vedette         localisationde dpt_code  \
27    PO7-00033       Aire                   none     none   
33    PO7-00039      Ajoie                   none     none   
35    PO7-00041      Ajoye                   none     none   
84    PO7-00097     Angeot  territoire de Belfort     none   
115   PO7-00132        Arc                   none     none   
...         ...        ...                    ...      ...   
3331  PO7-04074   Vézelois  territoire de Belfort     none   
3334  PO7-04077       Vézy                   none     none   
3337  PO7-04084  Vielverge              Côte-d’Or     none   
3446  PO7-04212     Vuache                   none     none   
3449  PO7-04226    Vully,                    none     none   

                         canton_code localisationco insee_code  
27                              none           none      08004  
33                              none           none       None  
35                              none           none       No

In [64]:
#On a fait ça le liage avec de l'exact match.

import pandas as pd

main_insee_commune = pd.read_csv("main_insee_commune.tsv", delimiter='\t')

# on utilise merge pour le liage avec dpt_code et dep_id
merged_df = pd.merge(filtered_df, main_insee_commune, left_on='dpt_code', right_on='DEP_id', how='left')

# on sélectionne les lignes où la colonne 'vedette' correspond à la colonne 'NCCENR' de main_insee_commune
matching_rows = merged_df.loc[merged_df['vedette'] == merged_df['NCCENR']]

# on crée une colonne 'insee_code' dans le dataframe filtered_df avec les codes correspondants à partir de main_insee_commune
filtered_df['insee_code'] = filtered_df.apply(lambda row: main_insee_commune.loc[main_insee_commune['NCCENR'] == row['vedette']]['insee_code'].values[0] if len(main_insee_commune.loc[main_insee_commune['NCCENR'] == row['vedette']]['insee_code'].values) > 0 else None, axis=1)


/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_9168/2876346075.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['insee_code'] = filtered_df.apply(lambda row: main_insee_commune.loc[main_insee_commune['NCCENR'] == row['vedette']]['insee_code'].values[0] if len(main_insee_commune.loc[main_insee_commune['NCCENR'] == row['vedette']]['insee_code'].values) > 0 else None, axis=1)


In [65]:
print(filtered_df)

     article_id      vedette localisationde dpt_code  \
7     PO7-00009     Abbenans          Doubs   DEP_25   
8     PO7-00010  Abbevillers          Doubs   DEP_25   
9     PO7-00011    Aboncourt    Haute-Saône   DEP_70   
10    PO7-00012    Abondance   Haute-Savoie   DEP_74   
11    PO7-00013       Abrets          Isère   DEP_38   
...         ...          ...            ...      ...   
3483  PO7-04280   Zaessingue      Haut-Rhin   DEP_68   
3484  PO7-04282   Zellenberg      Haut-Rhin   DEP_68   
3485  PO7-04285   Zillisheim      Haut-Rhin   DEP_68   
3486  PO7-04286   Zimmerbach      Haut-Rhin   DEP_68   
3487  PO7-04287  Zimmersheim      Haut-Rhin   DEP_68   

                       canton_code localisationco insee_code  
7              canton de Rougemont           none      25003  
8            canton d’Hérimoncourt           none       None  
9        canton de Combeaufontaine           none      54003  
10                            none           none      74001  
11    canton

In [72]:
#tous nos article_id qui correspondent à un code insee ; on peut donc en conclure que ce sont effectivement des 
#communes.

commune = filtered_df[filtered_df['insee_code'].notnull()]

In [74]:
#on a donc 2362 exact matches.

print(commune)

     article_id      vedette localisationde dpt_code  \
7     PO7-00009     Abbenans          Doubs   DEP_25   
9     PO7-00011    Aboncourt    Haute-Saône   DEP_70   
10    PO7-00012    Abondance   Haute-Savoie   DEP_74   
11    PO7-00013       Abrets          Isère   DEP_38   
12    PO7-00014     Accolans          Doubs   DEP_25   
...         ...          ...            ...      ...   
3483  PO7-04280   Zaessingue      Haut-Rhin   DEP_68   
3484  PO7-04282   Zellenberg      Haut-Rhin   DEP_68   
3485  PO7-04285   Zillisheim      Haut-Rhin   DEP_68   
3486  PO7-04286   Zimmerbach      Haut-Rhin   DEP_68   
3487  PO7-04287  Zimmersheim      Haut-Rhin   DEP_68   

                        canton_code localisationco insee_code  
7               canton de Rougemont           none      25003  
9         canton de Combeaufontaine           none      54003  
10                             none           none      74001  
11     canton du Pont-de-Beauvoisin           none      38001  
12    c

In [71]:
commune.to_csv("my_dataframe_commune.csv", index=False)

In [49]:
#les articles qui du coup ne seraient apparemment pas des communes.

pascommune = filtered_df.loc[filtered_df['insee_code'].isnull()]

In [75]:
#de ce que je vois, pour certains c'est une question d'appellation (par exemple, Aiguebellete ou Aillevillers, qui
#dans le référentiel insee ont d'autres noms: Aiguebelette-le-lac et Aillevillers-et-Lyaumont). Que faire? Essayer
#un regex pour reconnaître les chaînes de caractères similaires dans les vedettes? mais en faisant ça, on risque de
#lier ainsi tous les homonymes. Demander à Olivier?

#autre souci: certains d'entre eux ont des problèmes d'écriture (Burdigné -> Burdignes). Comment gérer ça? Faire à
#coup de fuzzy joins? Est-ce que ça pourrait marcher?

#en soustrayant les 116 articles sans dpt_code qui n'auraient pas pu être liés de toute manière, on se retrouve avec 
#333 articles avec dpt_code mais pas de insee_code qui matche.

print(pascommune)

     article_id         vedette localisationde dpt_code  \
8     PO7-00010     Abbevillers          Doubs   DEP_25   
17    PO7-00023    Aiguebelette         Savoie   DEP_73   
22    PO7-00028    Aillevillers    Haute-Saône   DEP_70   
33    PO7-00039           Ajoie           none     none   
35    PO7-00041           Ajoye           none     none   
...         ...             ...            ...      ...   
3438  PO7-04203           Voray    Haute-Saône   DEP_70   
3446  PO7-04212          Vuache           none     none   
3449  PO7-04226         Vully,            none     none   
3450  PO7-04227           Vyans    Haute-Saône   DEP_70   
3467  PO7-04253  Wihr-en-Plaine      Haut-Rhin   DEP_68   

                           canton_code localisationco insee_code  
8                canton d’Hérimoncourt           none       None  
17        canton du Pont-de-Beauvoisin           none       None  
22    canton de Saint-Loup-sur-Semouse           none       None  
33                     

In [51]:
pascommune.to_csv("my_dataframe_pas_commune.csv", index=False)